ПАКЕТ SURPRISE

    используйте данные MovieLens 1M
    можно использовать любые модели из пакета
    получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.


In [88]:
from surprise import KNNWithMeans, KNNBasic, SVD, BaselineOnly
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

import pandas as pd

In [89]:
# Dataset.load_builtin('ml-100k')

In [90]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [91]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [92]:
# movies_with_ratings[movies_with_ratings.userId == 2.0].title.unique()

In [93]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [94]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [95]:
# ratings.rating.min()

In [96]:
# ratings.rating.max()

In [97]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
# data_1 = Dataset.load_from_df(movies_with_ratings, reader)

In [98]:
# dataset[dataset.uid == 448].sort_values('rating', ascending=False)

In [99]:
# trainset, testset = train_test_split(data, test_size=.15)

In [100]:
kf = KFold(n_splits=5)

In [116]:
# algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
# algo = KNNBasic(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
# algo = SVD()
algo = BaselineOnly({'method': 'als', 'n_epochs': 5, 'reg_u': 12, 'reg_i': 5})
# algo = BaselineOnly({'method': 'sgd', 'learning_rate': .00005,})

In [117]:
# algo.fit(trainset)

In [118]:
# list(kf.split(data))

In [119]:
for trainset, testset in kf.split(data):
     # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

Estimating biases using als...
RMSE: 0.8680
Estimating biases using als...
RMSE: 0.8699
Estimating biases using als...
RMSE: 0.8722
Estimating biases using als...
RMSE: 0.8641
Estimating biases using als...
RMSE: 0.8624


In [120]:
# test_pred = algo.test(testset)

In [121]:
# accuracy.rmse(test_pred, verbose=True)

In [122]:
# algo.predict(uid=2, iid='Fight Club (1999)')

In [123]:
from surprise.model_selection import cross_validate

In [124]:
# algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})

In [125]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8735  0.8640  0.8627  0.8705  0.8645  0.8670  0.0042  
MAE (testset)     0.6729  0.6630  0.6641  0.6695  0.6638  0.6666  0.0039  
Fit time          0.16    0.15    0.14    0.14    0.15    0.15    0.01    
Test time         0.08    0.22    0.09    0.22    0.08    0.14    0.07    


{'test_rmse': array([0.87349045, 0.86398125, 0.86267806, 0.87051328, 0.86451301]),
 'test_mae': array([0.6728711 , 0.66301361, 0.66408967, 0.66945126, 0.66380871]),
 'fit_time': (0.16275978088378906,
  0.1498706340789795,
  0.14102649688720703,
  0.14300131797790527,
  0.14812183380126953),
 'test_time': (0.07825446128845215,
  0.22366905212402344,
  0.09063172340393066,
  0.22266459465026855,
  0.08053851127624512)}